<a href="https://colab.research.google.com/github/michel-wue/Seminar/blob/Tries_API_Data_extraction/Data_last_changes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import requests as r
import pandas as pd
from pandas import json_normalize
import re
from google.colab import drive
import datetime
from os import times

In [2]:
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
Data_basic = pd.DataFrame()

In [24]:
Data_basic = pd.DataFrame(columns=[
    'GameID','season','types',
    'week','HomeTeam_ID','AwayTeam_ID',
    'Winner'
    ])
week = list(range(1, 19))
seasons=list(range(2014,2023))

In [25]:
seasons

[2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022]

In [26]:

def infos(season,types,weeks):
  url = f"https://sports.core.api.espn.com/v2/sports/football/leagues/nfl/seasons/{season}/types/{types}/weeks/{weeks}/events"
  response = r.get(url)
  json_data = response.json()
  df = json_normalize(json_data['items'])
  for game in range(len(df)):
    l=[]
    response = r.get(df['$ref'][game])
    data=response.json()
    try:
      df2=json_normalize(data['competitions'])
      l.append(df2['id'][0])
      l.append(season)
      l.append(types)
      l.append(weeks)
      l.append(df2['competitors'][0][0]['id'])
      l.append(df2['competitors'][0][1]['id'])
      try:
        if df2['competitors'][0][0]['winner'] == True:
          l.append('Home')
        else:
          l.append('Away')
      except KeyError:
        l.append('NaN')
    except KeyError:
      print('it doesnt worked')
    Data_basic.loc[len(Data_basic)] = l


In [27]:
for x in seasons:  
  for i in week:
    infos(x,2,i)

In [29]:
Data_basic.to_csv('/content/drive/MyDrive/play_ID_data_2014-1022.csv', index=False)


In [28]:
print(Data_basic)





         GameID  season  types  week HomeTeam_ID AwayTeam_ID Winner
0     400554214    2014      2     1          26           9   Home
1     400554215    2014      2     1           1          18   Home
2     400554232    2014      2     1           3           2   Away
3     400554260    2014      2     1          12          10   Away
4     400554217    2014      2     1          14          16   Away
...         ...     ...    ...   ...         ...         ...    ...
2333  401437957    2022      2    18          21          19   Home
2334  401437962    2022      2    18          25          22   Home
2335  401437963    2022      2    18          26          14   Home
2336  401437959    2022      2    18          28           6   Home
2337  401437952    2022      2    18           9           8   Away

[2338 rows x 7 columns]


In [35]:
stats_df = pd.DataFrame()
stats_df = pd.DataFrame(columns=['GameID','clock','Pass_Incompletion_Home',
  'Punt_Home',
  'Timeout_Home',
  'Punt_Away',
  'Pass_Incompletion_Away',
  'Timeout_Away',
  'homeScore',
  'awayScore',
  'PassReception_Home_yards',
  'PassReception_Away_yards',
  'Penalty_Home',
  'Penalty_Away',
  'rushing_yards_home',
  'rushing_yards_away',
  'field_goals_good_home',
  'field_goals_good_away',
  'field_goals_missed_home',
  'field_goals_missed_away',
  'fumble_recovery_oponnent_home',
  'fumble_recovery_oponnent_away',
  'fumble_recovery_own_home',
  'fumble_recovery_own_away',
  'sack_home',
  'sack_away',
  'passing_TD_home',
  'passing_TD_away',
  'rushing_TD_home',
  'rushing_TD_away',
  'interception_home',
  'interception_away',
  'Interception_Return_TD_home',
  'Interception_Return_TD_away',
  'kick_off_return_TD_home',
  'kick_off_return_TD_away',
  'safety_home',
  'safety_away'])

In [40]:
def stats(GameID, Quarter, time):
    url = f'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{GameID}/competitions/{GameID}/drives?lang=en&region=us'
    response_drive = r.get(url)
    drive_data = response_drive.json()
    drive_df = json_normalize(drive_data['items'])
    data = []
    time = datetime.timedelta(seconds=time)
    clock = datetime.timedelta(seconds=0)
    Pass_Incompletion_Home = 0
    Punt_Home = 0
    Timeout_Home = 0
    Punt_Away = 0
    Pass_Incompletion_Away = 0
    Timeout_Away = 0
    homeScore = 0
    awayScore = 0
    PassReception_Home_yards = 0
    PassReception_Away_yards = 0
    Penalty_Home = 0
    Penalty_Away = 0
    rushing_yards_home = 0
    rushing_yards_away = 0
    field_goals_good_home = 0
    field_goals_good_away = 0
    field_goals_missed_home = 0
    field_goals_missed_away = 0
    fumble_recovery_oponnent_home = 0
    fumble_recovery_oponnent_away = 0
    fumble_recovery_own_home = 0
    fumble_recovery_own_away = 0
    passing_TD_home = 0
    passing_TD_away = 0
    rushing_TD_home = 0
    rushing_TD_away = 0
    interception_home = 0
    interception_away = 0
    Interception_Return_TD_home = 0
    Interception_Return_TD_away = 0
    kick_off_return_TD_home = 0
    kick_off_return_TD_away = 0
    safety_home = 0
    safety_away = 0
    sack_home = 0
    sack_away = 0
    for index, row in drive_df.iterrows():
        try:
            if (re.search(r"teams/(\d+)", row['team.$ref']).group(1) ==
                    Data_basic[Data_basic['GameID'] == GameID]['HomeTeam_ID'].values[0]):
                for detail in range(len(row['plays.items'])):
                    duration_str = row['plays.items'][detail]['clock']['displayValue']
                    duration_arr = duration_str.split(":")
                    duration = datetime.timedelta(minutes=int(duration_arr[0]), seconds=int(duration_arr[1]))
                    clock = datetime.timedelta(seconds=900) - duration
                    if row['plays.items'][detail]['period']['number'] <= Quarter and clock <= time:
                        if 'Penalty' in row['plays.items'][detail]['alternativeText'] or 'PENALTY' in row['plays.items'][detail]['alternativeText']:
                            if row['plays.items'][detail]['statYardage'] > 0:
                              Penalty_Away = Penalty_Away  + 1
                            elif row['plays.items'][detail]['statYardage'] < 0:
                              Penalty_Home = Penalty_Home + 1
                        if 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Pass Incompletion':
                            Pass_Incompletion_Home = Pass_Incompletion_Home + 1
                            PassReception_Home_yards = PassReception_Home_yards + row['plays.items'][detail][
                                'statYardage']
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Punt':
                            Punt_Home = Punt_Home + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Timeout':
                            Timeout_Home = Timeout_Home + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Pass Reception':
                            PassReception_Home_yards = PassReception_Home_yards + row['plays.items'][detail][
                                'statYardage']
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Rush':
                            rushing_yards_home = rushing_yards_home + row['plays.items'][detail]['statYardage']
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Sack':
                            sack_home = sack_home + 1
                            rushing_yards_home = row['plays.items'][detail]['statYardage'] + rushing_yards_home
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Passing Touchdown':
                            passing_TD_home = passing_TD_home + 1
                            PassReception_Home_yards = PassReception_Home_yards + row['plays.items'][detail][
                                'statYardage']
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Rushing Touchdown':
                            rushing_TD_home = rushing_TD_home + 1
                            rushing_yards_home = rushing_yards_home + row['plays.items'][detail]['statYardage']
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Pass Interception Return':
                            interception_home = interception_home + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Field Goal Good':
                            field_goals_good_home = field_goals_good_home + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Field Goal Missed':
                            field_goals_missed_home = field_goals_missed_home + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Interception Return Touchdown':
                            Interception_Return_TD_home = Interception_Return_TD_home + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Kickoff Return Touchdown':
                            kick_off_return_TD_home = kick_off_return_TD_home + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Fumble Recovery (Own)':
                            fumble_recovery_own_home = fumble_recovery_own_home + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Fumble Recovery (Opponent)':
                            fumble_recovery_oponnent_home = fumble_recovery_oponnent_home + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Safety':
                            safety_home = safety_home + 1
                        homeScore = row['plays.items'][detail]['homeScore']
            elif (re.search(r"teams/(\d+)", row['team.$ref']).group(1) !=
                  Data_basic[Data_basic['GameID'] == GameID]['HomeTeam_ID'].values[0]):
                for detail in range(len(row['plays.items'])):
                    duration_str2 = row['plays.items'][detail]['clock']['displayValue']
                    duration_arr2 = duration_str2.split(":")
                    duration2 = datetime.timedelta(minutes=int(duration_arr2[0]), seconds=int(duration_arr2[1]))
                    clock = datetime.timedelta(seconds=900) - duration2
                    if row['plays.items'][detail]['period']['number'] <= Quarter and clock <= time:
                        if 'Penalty' in row['plays.items'][detail]['alternativeText'] or 'PENALTY' in row['plays.items'][detail]['alternativeText']:
                            if row['plays.items'][detail]['statYardage'] > 0:
                              Penalty_Away = Penalty_Away  + 1
                            elif row['plays.items'][detail]['statYardage'] < 0:
                              Penalty_Home = Penalty_Home + 1
                        if 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Pass Incompletion':
                            Pass_Incompletion_Away = Pass_Incompletion_Away + 1
                            PassReception_Away_yards = PassReception_Away_yards + row['plays.items'][detail][
                                'statYardage']
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Punt':
                            Punt_Away = Punt_Away + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Timeout':
                            Timeout_Away = Timeout_Away + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Pass Reception':
                            PassReception_Away_yards = PassReception_Away_yards + row['plays.items'][detail][
                                'statYardage']
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Rush':
                            rushing_yards_away = rushing_yards_away + row['plays.items'][detail]['statYardage']
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Sack':
                            rushing_yards_away = row['plays.items'][detail]['statYardage'] + rushing_yards_away
                            sack_away = sack_away + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Passing Touchdown':
                            passing_TD_away = passing_TD_away + 1
                            PassReception_Away_yards = PassReception_Away_yards + row['plays.items'][detail][
                                'statYardage']
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Rushing Touchdown':
                            rushing_TD_away = rushing_TD_away + 1
                            rushing_yards_away = rushing_yards_away + row['plays.items'][detail]['statYardage']
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Pass Interception Return':
                            interception_away = interception_away + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Field Goal Good':
                            field_goals_good_away = field_goals_good_away + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Field Goal Missed':
                            field_goals_missed_away = field_goals_missed_away + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Interception Return Touchdown':
                            Interception_Return_TD_away = Interception_Return_TD_away + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Kickoff Return Touchdown':
                            kick_off_return_TD_away = kick_off_return_TD_away + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Fumble Recovery (Own)':
                            fumble_recovery_own_away = fumble_recovery_own_away + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Fumble Recovery (Opponent)':
                            fumble_recovery_oponnent_away = fumble_recovery_oponnent_away + 1
                        elif 'type' in row['plays.items'][detail] and row['plays.items'][detail]['type'][
                            'text'] == 'Safety':
                            safety_away = safety_away + 1
                        awayScore = row['plays.items'][detail]['awayScore']
        except TypeError:
            print('it doenst worked')
    data = [GameID, clock, Pass_Incompletion_Home,
            Punt_Home, Timeout_Home,
            Punt_Away, Pass_Incompletion_Away,
            Timeout_Away, homeScore,
            awayScore, PassReception_Home_yards,
            PassReception_Away_yards, Penalty_Home,
            Penalty_Away,
            rushing_yards_home,
            rushing_yards_away, field_goals_good_home,
            field_goals_good_away, field_goals_missed_home,
            field_goals_missed_away, fumble_recovery_oponnent_home,
            fumble_recovery_oponnent_away, fumble_recovery_own_home,
            fumble_recovery_own_away, sack_home,
            sack_away, passing_TD_home,
            passing_TD_away, rushing_TD_home,
            rushing_TD_away, interception_home,
            interception_away, Interception_Return_TD_home,
            Interception_Return_TD_away, kick_off_return_TD_home,
            kick_off_return_TD_away,
            safety_home, safety_away]
    stats_df.loc[len(stats_df)] = data




In [32]:
GameID='401437957'
url=f'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{GameID}/competitions/{GameID}/drives?lang=en&region=us'
response_drive = r.get(url)
drive_data = response_drive.json()
drive_df = json_normalize(drive_data['items'])
homescore=0
awayScore=0
Pass_Incompletion_Home=0
Punt_Home=0
Time_Out=0
rush = 0
penalty_home = 0
penalty_away = 0
plays = 0
for index, row in drive_df.iterrows():
  if (re.search(r"teams/(\d+)",row['team.$ref']).group(1) == Data_basic[Data_basic['GameID'] == GameID]['HomeTeam_ID'].values[0]):
          for detail in range(len(row['plays.items'])):
            plays = plays + 1
            #print(row['plays.items'][detail]['alternativeText'])
            if 'Penalty' in row['plays.items'][detail]['alternativeText'] or 'PENALTY' in row['plays.items'][detail]['alternativeText']:
              
              print(row['plays.items'][detail]['alternativeText'])
              #print(re.search(r"teams/(\d+)",row['plays.items'][detail]['team']['$ref']).group(1))
              print(row['plays.items'][detail]['statYardage'])
              if row['plays.items'][detail]['statYardage'] > 0:
                penalty_away = penalty_away  + 1
              elif row['plays.items'][detail]['statYardage'] < 0:
                penalty_home = penalty_home + 1
                print(penalty_home)
  elif (re.search(r"teams/(\d+)",row['team.$ref']).group(1) != Data_basic[Data_basic['GameID'] == GameID]['HomeTeam_ID'].values[0]):
          for detail in range(len(row['plays.items'])):
            plays = plays + 1
            #print(row['plays.items'][detail]['alternativeText'])
            if 'Penalty' in row['plays.items'][detail]['alternativeText'] or 'PENALTY' in row['plays.items'][detail]['alternativeText']:
              print(row['plays.items'][detail]['alternativeText'])
              print(row['plays.items'][detail]['statYardage'])

              #print(re.search(r"teams/(\d+)",row['plays.items'][detail]['team']['$ref']).group(1))
              if row['plays.items'][detail]['statYardage'] < 0:
                penalty_away = penalty_away  + 1
              elif row['plays.items'][detail]['statYardage'] > 0:
                penalty_home = penalty_home + 1
                
                
print(penalty_home)
print(penalty_away)


(1:47) (Shotgun) D.Webb scrambles right end pushed ob at PHI 46 for 8 yards (T.Edwards). PENALTY on PHI-T.Edwards, Unnecessary Roughness, 15 yards, enforced at PHI 46.
23
(12:05) J.Gillan punts 46 yards to PHI 35, Center-C.Kreiter. B.Covey ran ob at NYG 49 for 16 yards (L.Collins). PENALTY on NYG-L.Collins, Offensive Holding, 10 yards, enforced at NYG 49.
16
(10:07) (Shotgun) PENALTY on PHI-B.Graham, Neutral Zone Infraction, 5 yards, enforced at NYG 25 - No Play.
5
(7:28) (Shotgun) PENALTY on NYG-E.Neal, False Start, 5 yards, enforced at PHI 40 - No Play.
-5
(7:04) (Shotgun) D.Webb pass incomplete short right [H.Reddick]. PENALTY on NYG-D.Webb, Intentional Grounding, 16 yards, enforced at PHI 45.
-16
(2:00) (Shotgun) J.Hurts pass short right to A.Brown ran ob at NYG 36 for 11 yards. PENALTY on NYG-N.McCloud, Roughing the Passer, 15 yards, enforced at NYG 36.
26
(:19) (Shotgun) PENALTY on NYG-E.Neal, False Start, 5 yards, enforced at NYG 37 - No Play.
-5
(12:19) (Shotgun) J.Hurts pass s

In [19]:
GameID='401437957'
url=f'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/{GameID}/competitions/{GameID}/drives?lang=en&region=us'
response_drive = r.get(url)
drive_data = response_drive.json()
drive_df = json_normalize(drive_data['items'])
homescore=0
awayScore=0
Pass_Incompletion_Home=0
Punt_Home=0
Time_Out=0
rush = 0
penalty_home = 0
penalty_away = 0
plays = 0
for index, row in drive_df.iterrows():
  for detail in range(len(row['plays.items'])):
    print(row['plays.items'][detail]['alternativeText'])

print(penalty_home)
print(penalty_away)

J.Elliott kicks 63 yards from PHI 35 to NYG 2. G.Brightwell to NYG 26 for 24 yards (P.Johnson; K.Wallace).
(14:54) (Shotgun) M.Breida right guard to NYG 30 for 4 yards (K.White).
(14:18) (Shotgun) M.Breida left guard to NYG 33 for 3 yards (B.Graham; L.Joseph).
(13:42) (Shotgun) D.Webb pass incomplete short right.
(13:37) J.Gillan punts 46 yards to PHI 21, Center-C.Kreiter. B.Covey to PHI 35 for 14 yards (T.Fox).
(13:27) (Shotgun) J.Hurts pass short left to A.Brown to NYG 30 for 35 yards (C.Flott).
(13:00) (No Huddle, Shotgun) J.Hurts pass incomplete short right.
(12:55) (Shotgun) J.Hurts pass short left to Q.Watkins pushed ob at NYG 16 for 14 yards (N.McCloud).
(12:29) (No Huddle, Shotgun) J.Hurts pass short left to D.Goedert to NYG 14 for 2 yards (R.Williams).
(11:55) (Shotgun) J.Hurts pass incomplete short left to D.Goedert.
(11:50) (Shotgun) J.Hurts pass incomplete short left.
(11:43) J.Elliott 32 yard field goal is GOOD, Center-R.Lovato, Holder-B.Kern.
J.Elliott kicks 65 yards from

In [ ]:
#for plays in Data_basic['GameID']:
  #stats(plays,4,900)
  

KeyboardInterrupt: ignored

In [ ]:
#stats_df.to_csv('/content/drive/MyDrive/quarter1_data.csv', index=False)

In [41]:
stats('401437957',4,900)

In [42]:
stats_df

,GameID,clock,Pass_Incompletion_Home,Punt_Home,Timeout_Home,Punt_Away,Pass_Incompletion_Away,Timeout_Away,homeScore,awayScore,...,rushing_TD_home,rushing_TD_away,interception_home,interception_away,Interception_Return_TD_home,Interception_Return_TD_away,kick_off_return_TD_home,kick_off_return_TD_away,safety_home,safety_away
0,401437957,0 days 00:15:00,14,2,6,6,17,2,22,16,...,1,1,1,0,0,0,0,0,0,0
1,401437957,0 days 00:15:00,14,2,6,6,17,2,22,16,...,1,1,1,0,0,0,0,0,0,0


In [ ]:

url=f'http://sports.core.api.espn.com/v2/sports/football/leagues/nfl/events/401326316/competitions/401326316/drives?lang=en&region=us'
response_drive = r.get(url)
drive_data = response_drive.json()
drive_df = json_normalize(drive_data['items'])
 



In [ ]:
import datetime

duration_str = "15:00"
duration_arr = duration_str.split(":")
duration = datetime.timedelta(minutes=int(duration_arr[0]), seconds=int(duration_arr[1]))

print(duration)

In [ ]:
GameID='401326322'
for index, row in drive_df.iterrows():
  for detail in range(len(row['plays.items'])):
    #print(row['plays.items'][detail])

    duration_str = row['plays.items'][detail]['clock']['displayValue']
    duration_arr = duration_str.split(":")
    duration = datetime.timedelta(minutes=int(duration_arr[0]), seconds=int(duration_arr[1]))
    print(re.search(r"teams/(\d+)",row['plays.items'][detail]['team']['$ref']).group(1))
    if (datetime.timedelta(seconds=900)-duration) <= (datetime.timedelta(seconds=60)):
      print(datetime.timedelta(seconds=900)-duration)

In [ ]:
print(duration-datetime.timedelta(seconds=110))

In [ ]:
15*60